In [49]:
# %%writefile 'Colltable-from-timber.py'
import time
import pytimber
import pyoptics as opt
import numpy as np
import csv
import pandas as pd


mdb=pytimber.LoggingDB(source='mdb')

filln=5313
beam=2

OpticsDir = '/afs/cern.ch/user/n/nbiancac/ln_work/scratch0/IRIS/LHC_IW_model/Optics/2016/LHC-HighBeta/' # path for reference optics
CollDir='/afs/cern.ch/user/n/nbiancac/ln_work/scratch0/IRIS/LHC_IW_model/Coll_settings/'

newname = 'LHC_RP_highbeta_6.5TeV_B'+str(beam)

time_str1 = '2016-09-19_21:15:00' # 
time_str2 = '2016-09-19_21:40:00' # 
time_str3 = '2016-09-19_22:04:00' # 


scenario_scan=[{'time' : time_str},
              {'time' : time_str2},
              {'time' : time_str3}]

tfs = opt.optics.open((OpticsDir+'twiss_lhcb%d.tfs')%beam)
    
RP_names = tfs.name[tfs//'XRP*']
RP_betx = tfs.betx[tfs//'XRP*']
RP_bety = tfs.bety[tfs//'XRP*']

In [59]:
for scenario in scenario_scan:    
    
    en=3.5e-6 # reference normalized emittance
    gamma=tfs.param['gamma']; 
    
    print 'emittance = %.1f'%(en*1e6), 'gamma = %d'%gamma
        
    comment='fill_'+str(filln)+'_'+scenario['time']
    
    RP_hgap = []
    RP_phi = []
    RP_sigma =[]
    
    for RP_name in RP_names:
    
        var= mdb.search(RP_name+'%MEAS_LVDT_LU')[0]
        bety= tfs.bety[tfs//RP_name]
        betx= tfs.betx[tfs//RP_name]

        
        data=mdb.get(var,scenario['time'].replace('_',' '))
        tt,hgap=data[var]; tt+=2*3600
        hgap_new=hgap*1e-3;
        
        if 'XRPH' in RP_name:
            phi = 0.
        else:
            phi = np.pi/2
            
        sigma = np.sqrt(en/gamma*betx*np.cos(phi)**2+en/gamma*bety*np.sin(phi)**2)
        nsig = hgap_new/sigma
        
        RP_hgap.append(hgap_new)
        RP_phi.append(phi)
        RP_sigma.append(sigma)
        
        print RP_name, str(hgap_new)+' m', str(betx)+' m', str(bety)+' m '+str(nsig)+'\n',
    name_converted=newname+'_'+comment+'.txt'
    
    RP_phi = np.array(RP_phi).T
    RP_hgap = np.squeeze(np.array(RP_hgap).T)
    RP_sigma = np.squeeze(np.array(RP_sigma).T)
    
    
    
    
    lista = [[RP_names], [RP_hgap], [RP_betx], [RP_bety], [RP_phi], [RP_sigma]]

    raw_data = {'Name': RP_names,
        'Halfgap[m]': RP_hgap,
        'Betax[m]': RP_betx,
        'Betay[m]': RP_bety,
        'Thickness[m]': 10e-2*np.ones(np.shape(RP_hgap)),
        'Length[m]': 54e-3*np.ones(np.shape(RP_hgap)),
        'Material': ['ss304L' for ii in np.arange(len(RP_hgap))],
        'Angle[rad]': RP_phi,
        'nsigma': RP_sigma,
            }
    df = pd.DataFrame(raw_data, columns = raw_data.keys())
    df.to_csv(CollDir+name_converted,index=False,sep ='\t')
    print name_converted

emittance = 3.5 gamma = 6927
XRPV.B6L5.B2 [ 0.00299687] m [ 621.34040311] m [ 37.37614984] m [ 21.80862622]
XRPH.B6L5.B2 [ 0.00996868] m [ 635.59808002] m [ 37.13432605] m [ 17.59157539]
XRPH.E6L5.B2 [ 0.04131057] m [ 764.20215341] m [ 35.54001878] m [ 66.48372182]
XRPH.A6L5.B2 [ 0.04196228] m [ 786.53670836] m [ 35.35880868] m [ 66.56682595]
XRPV.A6L5.B2 [ 0.04328061] m [ 802.57471501] m [ 35.24425907] m [ 324.3451446]
XRPV.D6L5.B2 [ 0.00302117] m [ 862.08759559] m [ 34.92626861] m [ 22.74347247]
XRPH.D6L5.B2 [ 0.00999306] m [ 878.8767184] m [ 34.86541563] m [ 14.99659499]
XRPH.C6L5.B2 [ 0.04217176] m [ 1237.35846436] m [ 35.98731834] m [ 53.33744802]
XRPV.C6L5.B2 [ 0.00301767] m [ 1257.46522783] m [ 36.16364051] m [ 22.32513535]
XRPV.B7L1.B2 [ 0.00255895] m [ 216.05939707] m [ 37.63134954] m [ 18.55862728]
XRPV.A7L1.B2 [ 0.00303436] m [ 251.96921161] m [ 42.52425856] m [ 20.70171565]
LHC_RP_highbeta_6.5TeV_B2_fill_5313_2016-09-19_21:15:00.txt
emittance = 3.5 gamma = 6927
XRPV.B6L5.B2